# Game Logs
This notebook explores the use of these two endpoints in retrieving detailed game stats for a given player:
- https://v2.nba.api-sports.io/games
- https://v2.nba.api-sports.io/players/statistics

The code from this notebook will be used in V1 of the _Player Search_ feature.

In [1]:
from rel_bball_analytics.players.utils import get_data_from_api

from datetime import datetime
import pandas as pd

In [2]:
PLAYER_ID = 124
TEAM_ID = 11
SEASON = 2022

### Get Game Info from Team and Season

In [3]:
GAME_COLUMNS = [
    "id",
    "season",
    "date",
    "winner",
    "difference",
    "home",
    "away",
    "home_score",
    "away_score",
]

In [4]:
def parse_date(date):
    return datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.000Z")

In [5]:
def get_winner(row):
    return row["home"] if row["home_score"] > row["away_score"] else row["away"]

In [6]:
def filter_finished_games(games):
    games["is_finished"] = games["status"].apply(lambda obj: obj["long"] == "Finished")
    return games.loc[games.is_finished]

In [7]:
def clean_game_data(data):
    data["date"] = data["date"].apply(lambda obj: parse_date(obj.get("start")))
    data["home"] = data["teams"].apply(lambda obj: obj["home"].get("code"))
    data["away"] = data["teams"].apply(lambda obj: obj["visitors"].get("code"))
    data["home_score"] = data["scores"].apply(lambda obj: obj["home"].get("points"))
    data["away_score"] = data["scores"].apply(lambda obj: obj["visitors"].get("points"))
    data["difference"] = abs(data["home_score"] - data["away_score"])
    data["winner"] = data.apply(get_winner, axis=1)

    return data[GAME_COLUMNS]

In [8]:
params = {"team": TEAM_ID, "season": SEASON, "league": "Standard"}
game_data = get_data_from_api(endpoint="games", params=params)

if game_data["results"] == 0:
    print("No results found")

df_games = pd.DataFrame(game_data["response"])
df_games = filter_finished_games(games=df_games)
df_games = clean_game_data(data=df_games)

In [9]:
df_games.head()

,id,season,date,winner,difference,home,away,home_score,away_score
0,10980,2022,2022-09-30 10:00:00,GSW,9,WAS,GSW,87,96
1,10983,2022,2022-10-02 05:00:00,GSW,9,GSW,WAS,104,95
2,11019,2022,2022-10-10 00:30:00,LAL,3,GSW,LAL,121,124
3,11030,2022,2022-10-12 02:00:00,GSW,33,GSW,POR,131,98
4,11048,2022,2022-10-15 02:00:00,DEN,7,GSW,DEN,112,119
